In [211]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
from datetime import datetime, date
import os

## Read in Games and Game Details datasets

In [212]:
games = pd.read_csv('nba-games/games.csv')
game_details = pd.read_csv('nba-games/games_details.csv')
teams = pd.read_csv('nba-games/teams.csv')
teams.reset_index(drop=True)
teams.head()

,LEAGUE_ID,TEAM_ID,MIN_YEAR,MAX_YEAR,ABBREVIATION,NICKNAME,YEARFOUNDED,CITY,ARENA,ARENACAPACITY,OWNER,GENERALMANAGER,HEADCOACH,DLEAGUEAFFILIATION
0,0,1610612737,1949,2019,ATL,Hawks,1949,Atlanta,State Farm Arena,18729.0,Tony Ressler,Travis Schlenk,Lloyd Pierce,Erie Bayhawks
1,0,1610612738,1946,2019,BOS,Celtics,1946,Boston,TD Garden,18624.0,Wyc Grousbeck,Danny Ainge,Brad Stevens,Maine Red Claws
2,0,1610612740,2002,2019,NOP,Pelicans,2002,New Orleans,Smoothie King Center,NaN,Tom Benson,Trajan Langdon,Alvin Gentry,No Affiliate
3,0,1610612741,1966,2019,CHI,Bulls,1966,Chicago,United Center,21711.0,Jerry Reinsdorf,Gar Forman,Jim Boylen,Windy City Bulls
4,0,1610612742,1980,2019,DAL,Mavericks,1980,Dallas,American Airlines Center,19200.0,Mark Cuban,Donnie Nelson,Rick Carlisle,Texas Legends


## Merge game dataframes, drop rows with NAN values in OREB, PLUS_MINUS columns

In [213]:
games_df = pd.merge(games, game_details, on='GAME_ID', how='inner') 
games_df = games_df.dropna(how='any', subset=['OREB', 'PLUS_MINUS'])

#Create "Meta-data" Dataframe for game IDs, Team IDs, Team Names, Dates 
#games_meta = games[['GAME_DATE_EST', 'GAME_ID', 'HOME_TEAM_ID','VISITOR_TEAM_ID']].copy() 
#games_meta['HOME_TEAM'] = ' ' 
#games_meta['VISITOR_TEAM'] = ' ' 
#games_meta.head()

In [214]:
## Drop string-value columns. Re-check with logistic equivalents?
games_df.drop(['GAME_DATE_EST','TEAM_ABBREVIATION','TEAM_CITY', 'PLAYER_NAME','START_POSITION', 'MIN'], axis=1, inplace=True)
games_df.head()

,GAME_ID,GAME_STATUS_TEXT,HOME_TEAM_ID,VISITOR_TEAM_ID,SEASON,TEAM_ID_home,PTS_home,FG_PCT_home,FT_PCT_home,FG3_PCT_home,...,OREB,DREB,REB,AST,STL,BLK,TO,PF,PTS,PLUS_MINUS
0,41300405,Final,1610612759,1610612748,2013,1610612759,104.0,0.474,0.783,0.462,...,1.0,9.0,10.0,5.0,0.0,2.0,1.0,1.0,31.0,-16.0
1,41300405,Final,1610612759,1610612748,2013,1610612759,104.0,0.474,0.783,0.462,...,0.0,2.0,2.0,1.0,0.0,0.0,0.0,2.0,3.0,8.0
2,41300405,Final,1610612759,1610612748,2013,1610612759,104.0,0.474,0.783,0.462,...,0.0,7.0,7.0,2.0,1.0,0.0,0.0,2.0,13.0,-20.0
3,41300405,Final,1610612759,1610612748,2013,1610612759,104.0,0.474,0.783,0.462,...,0.0,3.0,3.0,1.0,0.0,0.0,3.0,3.0,11.0,-10.0
4,41300405,Final,1610612759,1610612748,2013,1610612759,104.0,0.474,0.783,0.462,...,1.0,4.0,5.0,2.0,1.0,0.0,4.0,2.0,5.0,-18.0


In [215]:
games.drop(['TEAM_ID_home','TEAM_ID_away', 'GAME_STATUS_TEXT'], axis=1, inplace=True)
games['GAME_DATE_EST'] = (pd.to_datetime(games['GAME_DATE_EST']))
games.head()

,GAME_DATE_EST,GAME_ID,HOME_TEAM_ID,VISITOR_TEAM_ID,SEASON,PTS_home,FG_PCT_home,FT_PCT_home,FG3_PCT_home,AST_home,REB_home,PTS_away,FG_PCT_away,FT_PCT_away,FG3_PCT_away,AST_away,REB_away,HOME_TEAM_WINS
0,2014-06-15,41300405,1610612759,1610612748,2013,104.0,0.474,0.783,0.462,25.0,40.0,87.0,0.400,0.741,0.280,14.0,41.0,1
1,2014-06-12,41300404,1610612748,1610612759,2013,86.0,0.451,0.650,0.409,13.0,27.0,107.0,0.571,0.720,0.429,25.0,44.0,0
2,2014-06-10,41300403,1610612748,1610612759,2013,92.0,0.516,0.750,0.476,17.0,26.0,111.0,0.594,0.813,0.450,21.0,29.0,0
3,2014-06-08,41300402,1610612759,1610612748,2013,96.0,0.439,0.600,0.462,26.0,37.0,98.0,0.529,0.762,0.421,16.0,38.0,0
4,2014-06-05,41300401,1610612759,1610612748,2013,110.0,0.588,0.773,0.520,30.0,39.0,95.0,0.474,0.818,0.414,16.0,29.0,1


In [216]:
home = input("Enter Home Team Abbreviation: ") 
visitor = input("Enter Visitor Team Abbreviation: ")
#game_date = input ("Enter Month, Year of Prediction Request (MM/DD/YYYY)")

for i in range(len(teams)):
    if teams.iloc[i]['ABBREVIATION'] == home:
        home_id = teams.iloc[i]['TEAM_ID']
    if teams.iloc[i]['ABBREVIATION'] == visitor:
        visitor_id = teams.iloc[i]['TEAM_ID']
        
#Convert game_date to datetime format    
#game_date = datetime.strptime(game_date, '%m/%d/%Y')

Enter Home Team Abbreviation: PHX
Enter Visitor Team Abbreviation: NYK


## Drop Rows to only teams and earlier dates specified

In [217]:
#games = games[games['GAME_DATE_EST'] <=game_date]
games = games[games['HOME_TEAM_ID'] == home_id]
games = games[games['VISITOR_TEAM_ID'] == visitor_id]

#Drop dates, team_ids, game_ids, season - will not be useful
games.drop(['GAME_DATE_EST','GAME_ID','HOME_TEAM_ID', 
            'VISITOR_TEAM_ID', 'SEASON'], axis=1, inplace=True)
games.dropna(inplace=True)

## Assign X and y data frames

In [218]:
X = games.drop("HOME_TEAM_WINS", axis=1)
y = games["HOME_TEAM_WINS"]
print(X.shape, y.shape) 

(15, 12) (15,)


## Use SciKitLearn to Split into Train / Test datasets 

In [219]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, stratify=y)

## SciKitLearn - Linear Regression

In [220]:
model = LinearRegression()

# Fitting our model with all of our features in X
model.fit(X, y)

score = model.score(X, y)
print(f"R2 Score: {score}")
predict = pd.DataFrame({"Prediction": model.predict(X_test), "Actual": y_test}).reset_index(drop=True)
contribution = pd.DataFrame({"Parameter": list(X.columns), "Weight": model.coef_ }).reset_index(drop=True)
predict


R2 Score: 0.8303525740627433


,Prediction,Actual
0,0.780869,1
1,1.147221,1
2,0.178748,0
3,0.812560,1


In [221]:
contribution.sort_values(by='Weight', ascending=False)

,Parameter,Weight
1,FG_PCT_home,8.358484
8,FT_PCT_away,2.092114
3,FG3_PCT_home,0.842893
9,FG3_PCT_away,0.627178
5,REB_home,0.023222
11,REB_away,0.010921
6,PTS_away,0.002685
0,PTS_home,-0.017775
4,AST_home,-0.056405
10,AST_away,-0.065656


## SciKitLearn - Logistic Regression

In [222]:
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression()
classifier

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [223]:
classifier.fit(X_train, y_train)

C:\Users\fireh\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [224]:
print(f"Training Data Score: {classifier.score(X_train, y_train)}")
print(f"Testing Data Score: {classifier.score(X_test, y_test)}")

Training Data Score: 1.0
Testing Data Score: 1.0


In [225]:
predictions = classifier.predict(X_test)
pd.DataFrame({"Prediction": predictions, "Actual": y_test}).reset_index(drop=True)

,Prediction,Actual
0,1,1
1,1,1
2,0,0
3,1,1
